In [44]:
import numpy as np
import random

class KMeans:
    """
    Minimal NumPy-only K-Means with functionized steps.
    No sklearn, no typing — pure Python version.

    Parameters
    ----------
    n_clusters : int
        Number of clusters (K).
    init : {"k-means++", "random"}
        Initialization scheme.
    max_iter : int
        Maximum iterations.
    tol : float
        Convergence tolerance on centroid shift (Euclidean norm).
    random_state : int or None
        Seed for reproducibility.
    """

    def __init__(self, n_clusters, init="k-means++", max_iter=300, tol=1e-4, random_state=None):
        self.n_clusters = int(n_clusters)
        self.init = init
        self.max_iter = int(max_iter)
        self.tol = float(tol)
        self.random_state = random_state

        # Attributes set after fitting
        self.cluster_centers_ = None
        self.labels_ = None
        self.inertia_ = None
        self.n_iter_ = None

    # ---------- core primitives ----------

    def _euclidean_squared(self, a, b):
        """
        Squared Euclidean distances between rows of a (n_samples, d)
        and rows of b (n_centroids, d). Returns (n_samples, n_centroids).
        """
        a2 = np.sum(a * a, axis=1, keepdims=True)
        b2 = np.sum(b * b, axis=1)
        ab = a @ b.T
        return a2 - 2.0 * ab + b2
    
    
    def _kmeanspp_init(self, X, rng):
        """
        k-means++ initialization. Returns (K, d) array of initial centroids.
        """
        n_samples, n_features = X.shape
        centroids = np.empty((self.n_clusters, n_features), dtype=X.dtype)

        # 1) choose first centroid uniformly
        idx0 = rng.integers(0, n_samples)
        centroids[0] = X[idx0]

        # 2) choose remaining with probability proportional to D^2
        closest_dist_sq = self._euclidean_squared(X, centroids[0:1]).ravel()
        for i in range(1, self.n_clusters):
            probs = closest_dist_sq / closest_dist_sq.sum()
            idx = rng.choice(n_samples, p=probs)
            centroids[i] = X[idx]
            d2 = self._euclidean_squared(X, centroids[i:i+1]).ravel()
            closest_dist_sq = np.minimum(closest_dist_sq, d2)
        return centroids

    # ---------- step-by-step methods ----------
    #rng = np.random.default_rng(42)
    def _initialize_centroids(self, X, rng=None):
        """Step 2: Initialize centroids randomly."""
        n_samples = X.shape[0]
        # Set Python random seed if random_state was provided
        if self.random_state is not None:
            random.seed(self.random_state)
        # Choose unique random indices for centroids
        indices = random.sample(range(n_samples), self.n_clusters)
        centroids = X[indices].copy()
        return centroids

    def _assign(self, X, centroids):
        """Step 3: Assignment — return (labels, d2)."""
        d2 = self._euclidean_squared(X, centroids)
        labels = np.argmin(d2, axis=1)
        return labels, d2

    def _update(self, X, labels, rng):
        """Step 4: Update — compute new centroids, handle empty clusters by re-seeding."""
        K, d = self.n_clusters, X.shape[1]
        new_centroids = np.empty((K, d), dtype=float)
        for j in range(K):
            mask = (labels == j)
            if np.any(mask):
                new_centroids[j] = X[mask].mean(axis=0)
            else:
                # reinitialize empty cluster to a random point
                new_centroids[j] = X[rng.integers(0, X.shape[0])]
        return new_centroids

    def _converged(self, old_centroids, new_centroids, old_labels, new_labels):
        """Step 5: Convergence check — label stability OR centroid shift < tol."""
        if old_labels is not None and np.array_equal(old_labels, new_labels):
            return True
        shift = float(np.sqrt(np.sum((new_centroids - old_centroids) ** 2)))
        return shift < self.tol

    def _compute_inertia(self, X, labels, centroids):
        """Step 6: Inertia (within-cluster SSE)."""
        return float(np.sum((X - centroids[labels]) ** 2))

    # ---------- public API ----------

    def fit(self, X):
        """
        Run K-Means on X.
        """
        X = np.asarray(X, dtype=float)
        rng = np.random.default_rng(self.random_state)

        centroids = self._initialize_centroids(X, rng)
        labels = None

        for it in range(1, self.max_iter + 1):
            new_labels, _ = self._assign(X, centroids)
            new_centroids = self._update(X, new_labels, rng)

            if self._converged(centroids, new_centroids, labels, new_labels):
                centroids = new_centroids
                labels = new_labels
                break

            centroids = new_centroids
            labels = new_labels

        # Final metrics
        inertia = self._compute_inertia(X, labels, centroids)

        # Set attributes
        self.cluster_centers_ = centroids
        self.labels_ = labels
        self.inertia_ = inertia
        self.n_iter_ = it
        return self

    def predict(self, X):
        """
        Assign cluster indices for new samples using fitted centroids.
        """
        if self.cluster_centers_ is None:
            raise RuntimeError("Model is not fitted. Call .fit(X) first.")
        X = np.asarray(X, dtype=float)
        labels, _ = self._assign(X, self.cluster_centers_)
        return labels

    def fit_predict(self, X):
        """
        Convenience: fit the model and return labels for X.
        """
        self.fit(X)
        return self.labels_


In [45]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans as SKKMeans
from sklearn.metrics import adjusted_rand_score

# ---- Load Iris dataset ----
iris = load_iris()
X = iris.data
y_true = iris.target

# ---- Your implementation ----
my_km = KMeans(n_clusters=3, init="k-means++", max_iter=300, tol=1e-4, random_state=42)
my_labels = my_km.fit_predict(X)

print("=== Your KMeans ===")
print(f"Iterations: {my_km.n_iter_}")
print(f"Inertia: {my_km.inertia_:.4f}")
print(f"ARI vs true labels: {adjusted_rand_score(y_true, my_labels):.4f}")

# ---- scikit-learn implementation ----
sk_km = SKKMeans(n_clusters=3, init="k-means++", n_init=1, max_iter=300, tol=1e-4,
                 algorithm="lloyd", random_state=42)
sk_labels = sk_km.fit_predict(X)

print("\n=== scikit-learn KMeans ===")
print(f"Iterations: {sk_km.n_iter_}")
print(f"Inertia: {sk_km.inertia_:.4f}")
print(f"ARI vs true labels: {adjusted_rand_score(y_true, sk_labels):.4f}")


=== Your KMeans ===
Iterations: 7
Inertia: 142.7541
ARI vs true labels: 0.4290

=== scikit-learn KMeans ===
Iterations: 11
Inertia: 78.8557
ARI vs true labels: 0.7163


In [46]:
# test_kmeans_standalone_for_user_class.py
# Standalone unit tests (no cornelltest) for a NumPy-only KMeans class.
# Assumes a class named `KMeans` is importable from the current environment
# (e.g., defined above in the same notebook/script, or from a module import).

import math
import numpy as np

# ---------------- Helper assertion functions ----------------

def assert_equals(expected, actual, msg=None):
    assert expected == actual, msg or f"Expected {expected!r}, got {actual!r}"

def assert_not_equals(a, b, msg=None):
    assert a != b, msg or f"Did not expect {a!r} == {b!r}"

def assert_true(expr, msg=None):
    assert bool(expr), msg or f"Expression is not true: {expr!r}"

def assert_false(expr, msg=None):
    assert not bool(expr), msg or f"Expression is not false: {expr!r}"

def assert_floats_equal(expected, actual, tol=1e-6, msg=None):
    if (isinstance(expected, float) and math.isnan(expected)) and (isinstance(actual, float) and math.isnan(actual)):
        return
    assert abs(expected-actual) <= tol, msg or f"Expected {expected}, got {actual} (tol={tol})"

def assert_float_lists_equal(expected, actual, tol=1e-6, msg=None):
    expected = np.asarray(expected, dtype=float)
    actual = np.asarray(actual, dtype=float)
    assert expected.shape == actual.shape, msg or f"Shape mismatch: {expected.shape} vs {actual.shape}"
    diff = np.abs(expected - actual)
    assert np.all(diff <= tol), msg or f"Max diff {diff.max()} exceeds tol {tol}"

def rows_equal_any_permutation(A, B, tol=1e-8):
    """Return True if A and B contain the same rows up to permutation (within tol)."""
    A = np.asarray(A, dtype=float)
    B = np.asarray(B, dtype=float)
    if A.shape != B.shape:
        return False
    used = np.zeros(B.shape[0], dtype=bool)
    for i in range(A.shape[0]):
        found = False
        for j in range(B.shape[0]):
            if not used[j] and np.allclose(A[i], B[j], atol=tol, rtol=0):
                used[j] = True
                found = True
                break
        if not found:
            return False
    return True

# ---------------- Tests for user's KMeans class ----------------

try:
    KMeans  # type: ignore
except NameError:
    try:
        from kmeans import KMeans  # try local module named kmeans.py
    except Exception as e:
        raise ImportError(
            "Could not find KMeans in the current environment. " "Define your KMeans class above or ensure 'from kmeans import KMeans' works."
        ) from e


def test_basic_shapes_and_types():
    rng = np.random.default_rng(0)
    X = np.vstack([
        rng.normal([0,0], 0.2, size=(30,2)),
        rng.normal([5,5], 0.2, size=(30,2)),
        rng.normal([10,0], 0.2, size=(30,2)),
    ])
    km = KMeans(n_clusters=3, init="random", random_state=123)
    labels = km.fit_predict(X)

    assert_equals(X.shape[0], labels.shape[0])
    assert_equals((3, X.shape[1]), km.cluster_centers_.shape)
    assert_true(km.inertia_ >= 0.0)
    assert_true(isinstance(km.n_iter_, int) and km.n_iter_ >= 1)


def test_determinism_with_seed():
    rng = np.random.default_rng(1)
    X = rng.normal(size=(100, 2))

    km1 = KMeans(n_clusters=3, init="random", random_state=7)
    km2 = KMeans(n_clusters=3, init="random", random_state=7)

    labels1 = km1.fit_predict(X)
    labels2 = km2.fit_predict(X)

    assert_float_lists_equal(labels1, labels2, tol=0.0)
    assert_true(rows_equal_any_permutation(km1.cluster_centers_, km2.cluster_centers_))


def test_predict_matches_training_assignment():
    rng = np.random.default_rng(2)
    X = np.vstack([rng.normal([0,0], 0.3, size=(50,2)),
                   rng.normal([4,4], 0.3, size=(50,2))])
    km = KMeans(n_clusters=2, init="random", random_state=9).fit(X)

    d2 = ((X[:, None, :] - km.cluster_centers_[None, :, :])**2).sum(axis=2)
    nearest = np.argmin(d2, axis=1)
    assert_float_lists_equal(nearest, km.labels_, tol=0.0)

    new_pts = np.array([[0.05, -0.02], [4.1, 3.9]])
    preds = km.predict(new_pts)
    d2_new = ((new_pts[:, None, :] - km.cluster_centers_[None, :, :])**2).sum(axis=2)
    nearest_new = np.argmin(d2_new, axis=1)
    assert_float_lists_equal(preds, nearest_new, tol=0.0)


def test_inertia_definition():
    rng = np.random.default_rng(3)
    X = rng.normal(size=(80, 3))
    km = KMeans(n_clusters=4, init="random", random_state=11).fit(X)

    d2 = ((X[:, None, :] - km.cluster_centers_[None, :, :])**2).sum(axis=2)
    manual_inertia = float(np.sum(d2[np.arange(X.shape[0]), km.labels_]))
    assert_floats_equal(km.inertia_, manual_inertia, tol=1e-8)


def run_all_tests():
    print("Running KMeans standalone tests...")
    test_basic_shapes_and_types()
    print("  \u2713 test_basic_shapes_and_types")
    test_determinism_with_seed()
    print("  \u2713 test_determinism_with_seed")
    test_predict_matches_training_assignment()
    print("  \u2713 test_predict_matches_training_assignment")
    test_inertia_definition()
    print("  \u2713 test_inertia_definition")
    print("All tests passed!")

if __name__ == '__main__':
    run_all_tests()

Running KMeans standalone tests...
  ✓ test_basic_shapes_and_types
  ✓ test_determinism_with_seed
  ✓ test_predict_matches_training_assignment
  ✓ test_inertia_definition
All tests passed!


Absolutely ✅ — here’s your content rewritten in **Markdown (MD)** format, ready to drop directly into a Jupyter Notebook section titled:

---

# 🧩 K-Means Clustering

## What is Cluster Analysis?

Cluster analysis is the process of **grouping together similar points into clusters**.
A *point* can have 2, 3, or even hundreds of dimensions — meaning it’s a vector in some space.

One practical example is **epidemiological clustering**:
you might have 2D points representing the **longitude and latitude** of locations where birds carrying different strains of avian flu were found.
By clustering these points, you can gain insight into which regions correspond to each strain.

---

## Distances Between Points

To cluster data, we must measure **how close or far** points are from each other.
The **Euclidean distance** is the most common measure.

For two 2D points:
[
d(p, q) = \sqrt{(p_x - q_x)^2 + (p_y - q_y)^2}
]

For two 3D points:
[
d(p, q) = \sqrt{(p_x - q_x)^2 + (p_y - q_y)^2 + (p_z - q_z)^2}
]

In general, for two ( n )-dimensional points
( p = (p_1, p_2, …, p_n) ) and ( q = (q_1, q_2, …, q_n) ):
[
d(p, q) = \sqrt{(p_1 - q_1)^2 + (p_2 - q_2)^2 + \cdots + (p_n - q_n)^2}
]

**Example:**
[
p = (0.1, 0.2, 0.3, 0.4), \quad q = (0.0, 0.2, 0.3, 0.2)
]
[
d(p, q) = \sqrt{(0.1-0.0)^2 + (0.2-0.2)^2 + (0.3-0.3)^2 + (0.4-0.2)^2} = 0.7071…
]

---

## Cluster Centroids

The **centroid** of a cluster is its *center of mass* — the **average position** of all points in that cluster.

Even though it’s the mean of all cluster points, the centroid does **not** need to be one of those points.

To compute a centroid:
[
\text{centroid} = \frac{1}{N} \sum_{i=1}^{N} p_i
]

For points:
[
p + q = (p_1 + q_1, p_2 + q_2, …, p_n + q_n)
]
and dividing by a scalar ( a ):
[
\frac{p}{a} = \left(\frac{p_1}{a}, \frac{p_2}{a}, …, \frac{p_n}{a}\right)
]

---

## The K-Means Algorithm

The **idea** behind K-Means is simple:
each point belongs to the cluster whose centroid (mean) it is **closest** to.

However, because centroids depend on which points are assigned to them, and the assignments depend on the centroids — we solve this **chicken-and-egg** problem iteratively.

---

### Step 1. Pick ( k ) Centroids

We start by choosing ( k ), the number of clusters we want.

Each centroid ( m_j ) is an ( n )-dimensional point:
[
m_j = (m_{j,1}, m_{j,2}, …, m_{j,n})
]

We randomly select ( k ) points from the dataset as our **initial centroids**.
This is known as the **Forgy Method**, and is one of the most common ways to initialize k-means.

---

### Step 2. Partition the Dataset

Each point is assigned to the **nearest centroid** based on Euclidean distance.

If a point is equally close to two or more centroids, we break ties arbitrarily (usually by index order).

This produces ( k ) sets ( S_1, S_2, …, S_k ),
where each set ( S_j ) contains all points closest to centroid ( m_j ).

---

### Step 3. Recompute the Means

For each cluster ( S_j ), we recompute its centroid (mean):
[
m_j = \frac{1}{|S_j|} \sum_{q \in S_j} q
]

That is, we take the average of all points assigned to that cluster.

Since centroids have changed, some points may now be closer to a different centroid —
so we **repeat the assignment step**.

---

### Step 4. Repeat Until Convergence

We alternate between **assigning points** and **recomputing centroids** until:

* The centroids stop moving, or
* The assignments no longer change.

This means the algorithm has **converged**.

Sometimes convergence takes too long (many iterations).
Therefore, we often set a **maximum iteration limit** to prevent infinite loops.

---

### Summary of the K-Means Algorithm

1. Choose ( k ) and initialize centroids randomly.
2. Assign each point to the nearest centroid.
3. Recompute each centroid as the mean of its assigned points.
4. Repeat steps 2–3 until centroids stop changing or max iterations reached.

---

✅ **Key Idea:**
K-Means minimizes the **within-cluster sum of squared errors (inertia)** —
that is, the total squared distance between points and their assigned centroids.

---

Would you like me to add a small **Matplotlib visualization example** at the end of this Markdown (e.g., showing how centroids move across iterations on a 2D dataset)?


### For particle physics, what tricks you can come up to reduce the computation complexity of number and coordination for the centroids

### comparison of computational complexity for elbow and silhouette